In [2]:
#!pip install -r "requirements.txt"

In [181]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [77]:
pd.options.display.max_columns = None

In [156]:
pokemon = pd.read_csv("./pokemon.csv", index_col="#")
combats = pd.read_csv("./combats.csv")

In [157]:
pokemon.head()

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [158]:
combats.head()

,Winner,First_pokemon,Second_pokemon
0,650,650,579
1,151,151,558
2,340,340,631
3,139,688,139
4,132,365,132


In [159]:
combats['has_first_pokemon_win'] = combats['Winner'] == combats['First_pokemon']

In [163]:
combats.loc[combats['has_first_pokemon_win'] == True, 'Winner'] = 1
combats.loc[combats['has_first_pokemon_win'] == False, 'Winner'] = 2
combats.drop(columns=['has_first_pokemon_win'], inplace=True)

In [164]:
combats.head()

,Winner,First_pokemon,Second_pokemon
0,1,650,579
1,1,151,558
2,1,340,631
3,2,688,139
4,2,365,132


In [165]:
pokemon = pokemon.drop(columns='Name')

In [166]:
encoder = LabelEncoder()
categorical_cols = ['Type 1', 'Type 2']
for col in categorical_cols:
    pokemon[col] = encoder.fit_transform(pokemon[col])

In [167]:
pokemon.Legendary = pokemon.Legendary.map(lambda el:  1 if el == 'True' else 0)

In [168]:
pokemon.head()

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,
1,9,13,45,49,49,65,65,45,1,0
2,9,13,60,62,63,80,80,60,1,0
3,9,13,80,82,83,100,100,80,1,0
4,9,13,80,100,123,122,120,80,1,0
5,6,18,39,52,43,60,50,65,1,0


In [185]:
result1 = pd.merge(combats, pokemon.add_prefix("First_pokemon_"), how="left", left_on=['First_pokemon'], right_index=True)
result2 = pd.merge(result1, pokemon.add_prefix("Second_pokemon_"), how="left", left_on=['Second_pokemon'], right_index=True)

In [186]:
X = result2.drop(columns=['Winner', 'First_pokemon', 'Second_pokemon'])
y = result2.Winner

In [190]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

In [194]:
model = RandomForestClassifier(random_state=20)
model.fit(X_train, y_train)

print("Trained!")

Trained!


In [196]:
train_score = model.score(X_train, y_train)
print(f"Train set score: {round(train_score, 2) * 100}%")

Train set score: 100.0%


In [197]:
test_score = model.score(X_test, y_test)
print(f"Train set score: {round(test_score, 2) * 100}%")

Train set score: 95.0%


In [199]:
y_train

27434    2
13895    1
38835    1
30654    1
12278    1
        ..
31962    1
23452    1
23775    2
37135    1
27098    2
Name: Winner, Length: 30000, dtype: int64